## TASK:
<b>1. Implement Dijkstra algorithm to find shortest paths in graph between two vertices.</b><br>
<b>2. Find the number of shortest paths between two vertices</b><br>
<br>
<b>Input file format:</b><br> 
First line contains number of vertices and edges<br>
Other lines contain edges: start vertice, end vertice, edge weight.<br>
Vertices enumeration starts with 1 <br>
Graph may contain multiple edges.<br>
<br>

Task graph taken from test files for [9th DIMACS Implementation Challenge - Shortest Paths](http://www.diag.uniroma1.it//challenge9/download.shtml). It represents a road map of Florida, USA.

In [53]:
import time
import itertools
import numpy as np
import collections
import math

In [54]:
def read_graph_from_file(filename): #read edges from file into array
    with open(f'{filename}.txt', 'r') as f:
        E = []
        for line in f:
            l = line.split()
            l_1 = [int(item) for item in l]
            E.append(l_1)
    return E

def read_matrix_from_file(filename): 
    with open(f'{filename}.txt', 'r') as f:
        M = []
        for line in f:
            l = line.split()
            l_1 = []
            for item in l:
                if item == '--':
                    l_1.append(np.inf)
                else:
                    l_1.append(int(item))
            M.append(l_1)
    return M

def remove_multiple_edges(E): #прибираємо кратні ребра
    E.sort()
    res = list(E for E,_ in itertools.groupby(E))
    return res

def edges_to_dict(e, V): # create adjacency list from list of edges
    edict = {}
    e_sorted = sorted(e, key=lambda x: x[0])
    for item in e_sorted:
        if item[0] not in edict:
            edict[item[0]] = [(item[1], item[2])]
        else:
            edict[item[0]].append((item[1], item[2]))
    for v in V:
        if v not in edict:
            edict[v] = None
    return edict

In [55]:
def print_dict_values(dic):
    i = 1
    for row in dic:
        print(f"{i}:",list(row.values()))
        i += 1
        
def print_matr(matr):
    for row in matr:
        print(row)
        
def compare_dict_matr(dic, matr):
    for i in range(len(matr)):
        if matr[i] != list(dic[i].values()):
            print("WRONG")
            print("matr[i]=", matr[i])
            print("dictval=", list(dic[i].values()))
            break
    print("RIGHT")

In [56]:
##### MIN HEAP AS PRIORITY QUEUE #####
#### Heap items: (vertex, priority). Top item is one with min priority #####

class MinHeap:
    def __init__(self):
        self.heap = []
        
    def get_len(self):
        return len(self.heap)
    
    def parent(self, x):
        if x==0:
            return None
        else: 
            ind = int((x-1)/2)
            return ind

    def left(self, x):
        ind = 2*x+1
        if ind >= len(self.heap):
            return None
        else:
            return ind

    def right(self, x):
        ind = 2*x + 2
        if ind >= len(self.heap):
            return None
        else:
            return ind
        
    def print_heap(self):
        j = 0
        H = math.ceil(math.log(len(self.heap)+1,2))
        for i in range(H):
            fin = j*2 +1
            if fin < len(self.heap):
                print(self.heap[j:fin])
                j = fin
            else: 
                print(self.heap[j:])
                
    def heap_minimum(self): #A-min heap
        return self.heap[0]

    def min_heapify(self, x=0):
        l = self.left(x)
        r = self.right(x)
        if not r == l == None:            
            if l != None:
                if l <= len(self.heap) and self.heap[l][1] < self.heap[x][1]:
                    smallest = l
                else: 
                    smallest = x
            if r != None:
                if r <= len(self.heap) and self.heap[r][1] < self.heap[smallest][1]:
                    smallest = r
            if smallest != x:
                self.heap[x], self.heap[smallest] = self.heap[smallest], self.heap[x]
                self.min_heapify(smallest)
                
    def heap_increase_key_min(self):
        i = len(self.heap) - 1
        while i > 0 and self.heap[self.parent(i)][1] > self.heap[i][1]:
            self.heap[i], self.heap[self.parent(i)] = self.heap[self.parent(i)], self.heap[i]
            i = self.parent(i)

    def pop_heap_minimum(self):
        x = self.heap_minimum()
        self.heap.pop(0)
        self.min_heapify(0)
        return x
        
    def add_element(self, x):
        if len(self.heap) == 0:
            self.heap.append(x)
        else:
            self.heap.append(x)
            self.heap_increase_key_min()

In [57]:
class Graph_i: #graph operations done iteratively
    def __init__(self, V, E, adjacency_list):
        self.V = V
        self.E = E
        self.adj = adjacency_list
        self.fdict = {}
        
        
    def Dijkstra(self, start, end): 
        Q = MinHeap() #Priority Queue vertices and min path lengths towards them
        n = len(self.V)
        in1 = np.full((n), np.inf)
        in2 = [[item] for item in np.full((n), None)]
        A = dict(zip(self.V, in1)) #dict of (vertex, len of shortest path to it from the start vertex)
        A[start] = 0
        B = dict(zip(self.V, in2)) #dict of (vertex, its previous vertex)
        Q.add_element([start, A[start]])
        X = [start]
        v = start
        while Q.get_len() != 0:
            v = Q.pop_heap_minimum()[0]
            if self.adj[v] != None:
                for item in self.adj[v]:
                    u = item[0]
                    w = item[1]
                    if A[u] > A[v] + w:
                        A[u] = A[v] + w
                        B[u] = [v]
                        Q.add_element([u, A[u]])
                    elif (A[u] == A[v] + w) and v not in B[u]:
                        B[u].append(v)
            if v == end:
                print(f"shortest path len from {start} to {end} =", A[v])
        return A, B
    
    def Dijxtra_matrix(self, end): #generate matrix with shortest paths lengths from every vertex to all other
        M = []
        for v in V:
            A = self.Dijkstra(v, end)[0]
            M.append(A)
        return M
    
    def run_back(self, start, v, B): #run through all predecessors and check if we can get to the start vertex
        global counter
        for u in B[v]:
            if u != start:
                self.run_back(start, u, B)
            else: 
                counter += 1
        return counter

        
    def find_unique_paths(self, start, end):#find the number of unique shortest paths between start and end vertices
        A, B = self.Dijkstra(start, end)
#         print("A=", A)
#         print("B=", B)
        c = self.run_back(start, end, B)
        print("unique sortest paths=", c)

In [58]:
# E = read_graph_from_file("input_5_10")
# E = read_graph_from_file("test_paths")
E = read_graph_from_file("USA-FLA")
n = E[0][0]
E.pop(0)

print("n=", n) #number of graph vertices
E = remove_multiple_edges(E)
V = [i for i in range(1,n+1)]
adj = edges_to_dict(E, V) #Adjacency list 
# print("adj=", adj)

n= 1070376


In [59]:
G = Graph_i(V, E, adj)

start = time.time()
counter = 0
G.find_unique_paths(100562, 1070345)
end = time.time()
print("time=", end - start)

shortest path len from 100562 to 1070345 = 6699685
unique sortest paths= 4
time= 24.33692765235901
